In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import pycountry
import wrangle 

In [2]:
df = pd.read_csv("./Unicorn_Companies.csv")
df.head()
#print(df.shape)

,Company,Valuation ($B),Date Joined,Country,City,Industry,Select Inverstors,Founded Year,Total Raised,Financial Stage,Investors Count,Deal Terms,Portfolio Exits
0,Bytedance,$140,4/7/2017,China,Beijing,Artificial intelligence,"Sequoia Capital China, SIG Asia Investments, S...",2012.0,$7.44B,IPO,28.0,8.0,5.0
1,SpaceX,$100.3,12/1/2012,United States,Hawthorne,Other,"Founders Fund, Draper Fisher Jurvetson, Rothen...",2002.0,$6.874B,NaN,29.0,12.0,NaN
2,Stripe,$95,1/23/2014,United States,San Francisco,Fintech,"Khosla Ventures, LowercaseCapital, capitalG",2010.0,$2.901B,Asset,39.0,12.0,1.0
3,Klarna,$45.6,12/12/2011,Sweden,Stockholm,Fintech,"Institutional Venture Partners, Sequoia Capita...",2005.0,$3.472B,Acquired,56.0,13.0,1.0
4,Epic Games,$42,10/26/2018,United States,Cary,Other,"Tencent Holdings, KKR, Smash Ventures",1991.0,$4.377B,Acquired,25.0,5.0,2.0


In [3]:
df["Select Inverstors"][df["Select Inverstors"].isnull()]

10      NaN
217     NaN
292     NaN
318     NaN
438     NaN
639     NaN
757     NaN
789     NaN
814     NaN
882     NaN
911     NaN
916     NaN
928     NaN
939     NaN
951     NaN
963     NaN
1014    NaN
Name: Select Inverstors, dtype: object

In [4]:
#ind_shifted.remove(789)
ind_shifted = df[df["Select Inverstors"] == "NaN"].index.to_list()
df.iloc[ind_shifted, :]

,Company,Valuation ($B),Date Joined,Country,City,Industry,Select Inverstors,Founded Year,Total Raised,Financial Stage,Investors Count,Deal Terms,Portfolio Exits


In [5]:
ind_shifted = df[df["Select Inverstors"].isnull()].index.to_list()
ind_shifted.remove(789)
for i, row in df.iterrows():
    if i in ind_shifted:
        city, industry, invest = df.at[i, "City"], df.at[i, "Industry"], df.at[i, "Select Inverstors"]
        df.at[i, "City"] = invest
        df.at[i, "Industry"] = city
        df.at[i, "Select Inverstors"] = industry

##date and valuation 
df["Valuation ($B)"] = df["Valuation ($B)"].apply(lambda x: x[1:]).astype(float)
df["Date Joined"] = pd.to_datetime(df["Date Joined"])
##
# fill "None" values with np.nan
def replace_none_with_npnan(x):
    return np.nan if x == "None" else x

df = df.applymap(replace_none_with_npnan)
df["Investors Count"] = df["Investors Count"].fillna(0)
#df.info()
######
# Get the actual values of "Total Raised" and replace the wrong values in data
def get_actual_total_raised(value: str) -> float:
    to_replace = "$BMK"
    unity = value[-1]
    value = float(''.join(v for v in value if v not in to_replace))
    if unity == "B":
        result = value * 1000000000
    elif unity == "M":
        result = value * 1000000
    elif unity == "K":
        result = value * 1000
    else:
        result = value
    return result
   
df["Total Raised"] = df["Total Raised"].apply(lambda x: get_actual_total_raised(x) if x is not np.nan else x)
df["Total Raised"] = df["Total Raised"] / 1e6
df = df.rename(columns={"Total Raised" : "Total Raised(M)", 
                        "Select Inverstors" : "Select Investors"})

df['Financial Stage'] = df['Financial Stage'].replace({"Acq" : "Acquired"})
df["Industry"] = df["Industry"].replace({"Finttech" : "Fintech", 
                                         "Artificial intelligence" : "Artificial Intelligence"})
#########
# change column types
df["Country"] = df["Country"].astype('category')
df["City"] = df["City"].astype('category')
df["Industry"] = df["Industry"].astype('category')
df["Investors Count"] = df["Investors Count"].astype(int)

   

In [6]:
#cleanded df
#df

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1037 entries, 0 to 1036
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Company           1037 non-null   object        
 1   Valuation ($B)    1037 non-null   float64       
 2   Date Joined       1037 non-null   datetime64[ns]
 3   Country           1037 non-null   category      
 4   City              1021 non-null   category      
 5   Industry          1037 non-null   category      
 6   Select Investors  1036 non-null   object        
 7   Founded Year      994 non-null    float64       
 8   Total Raised(M)   1013 non-null   float64       
 9   Financial Stage   49 non-null     object        
 10  Investors Count   1037 non-null   int32         
 11  Deal Terms        1008 non-null   float64       
 12  Portfolio Exits   49 non-null     float64       
dtypes: category(3), datetime64[ns](1), float64(5), int32(1), object(3)
memory usag

In [8]:
# df_with_fyear = df[~df['Founded Year'].isna()]
# num_by_founded_year = df_with_fyear["Founded Year"].value_counts().reset_index()
# num_by_founded_year

In [9]:
df_with_fyear = df[~df['Founded Year'].isna()]
#df_with_fyear=df_with_fyear[df_with_fyear['Industry']=="Other"]
num_by_founded_year = df_with_fyear["Founded Year"].value_counts().reset_index()
num_by_founded_year.columns=["index", "Founded Year"]
num_by_founded_year["index"] = num_by_founded_year["index"].astype(np.int64)
num_by_founded_year = num_by_founded_year[num_by_founded_year["index"] >= 1990]
num_by_founded_year.sort_values(by=["index"], inplace=True)
years = pd.DataFrame({"years" : num_by_founded_year["index"]})

fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=num_by_founded_year["index"], y=num_by_founded_year["Founded Year"],
                            mode='lines',
                           name='lines')
               )
#from here we can neglect it or hash it 
# updatemenu = []
# buttons = []

# buttons.append(dict(method='update',
#                          label="All industries",
#                          visible=True,
#                          args=[{'y':[num_by_founded_year["Founded Year"]],
#                                 'x':[num_by_founded_year["index"]],
#                                 'type':'scatter'},
#                               {'title': "Number of unicorns in All industries since 1990"}],
#                         )
#                    )

# for indst in df.Industry.unique():
#     selected_industry = df_with_fyear[df_with_fyear["Industry"] == indst]
#     temp_vc = selected_industry["Founded Year"].value_counts().reset_index()
#     temp_vc.columns=["index", "Founded Year"]
#     temp_vc["index"] = temp_vc["index"].astype(np.int64)
#     temp_vc.sort_values(by=["index"], inplace=True)
#     result = years.set_index('years').join(temp_vc.set_index('index'), how='left').fillna(0)
#     result["Founded Year"] = result["Founded Year"].astype(int)
#     buttons.append(dict(method='update',
#                         label=indst,
#                         visible=True,
#                         args=[{'y':[result["Founded Year"]],
#                                'x':[years["years"]],
#                                'type':'scatter'},
#                                {'title': f"Number of unicorns in {indst} since 1990"}],
#                         )
#                   )

# updatemenu = [dict()]

# updatemenu[0]['buttons'] = buttons
# updatemenu[0]['direction'] = 'down'
# updatemenu[0]['showactive'] = True

# fig1.update_layout(showlegend=False, updatemenus=updatemenu)
# fig1.update_layout(
#     title = "Number of unicorns in All industries since 1990",
#     xaxis_title='Founded Year',
#     yaxis_title='Unicorns count')
# fig1.show()

In [59]:
#china or united state or sweeden
investors = []
for i, row in df.iterrows():
    if row["Select Investors"] is not np.nan:
        investors += row["Select Investors"].split(', ')
investors = pd.Series(investors).value_counts()[:10]
investors.sort_values(ascending=True, inplace=True)

fig2 = go.Figure([go.Bar(x=investors.values, y=investors.index, orientation='h')])
fig2.update_layout( # we can see top investor in all counteie or china and set it as variable
    title = "Top 10 investors",
    xaxis_title='Unicorns count',
    yaxis_title='Investors')
fig2.show()

In [11]:
df.sort_values("Valuation ($B)", ascending=False)[:20]


,Company,Valuation ($B),Date Joined,Country,City,Industry,Select Investors,Founded Year,Total Raised(M),Financial Stage,Investors Count,Deal Terms,Portfolio Exits
0,Bytedance,140.0,2017-04-07,China,Beijing,Artificial Intelligence,"Sequoia Capital China, SIG Asia Investments, S...",2012.0,7440.00,IPO,28,8.0,5.0
1,SpaceX,100.3,2012-12-01,United States,Hawthorne,Other,"Founders Fund, Draper Fisher Jurvetson, Rothen...",2002.0,6874.00,NaN,29,12.0,NaN
2,Stripe,95.0,2014-01-23,United States,San Francisco,Fintech,"Khosla Ventures, LowercaseCapital, capitalG",2010.0,2901.00,Asset,39,12.0,1.0
3,Klarna,45.6,2011-12-12,Sweden,Stockholm,Fintech,"Institutional Venture Partners, Sequoia Capita...",2005.0,3472.00,Acquired,56,13.0,1.0
4,Epic Games,42.0,2018-10-26,United States,Cary,Other,"Tencent Holdings, KKR, Smash Ventures",1991.0,4377.00,Acquired,25,5.0,2.0
5,Canva,40.0,2018-01-08,Australia,Surry Hills,Internet software & services,"Sequoia Capital China, Blackbird Ventures, Mat...",2012.0,571.26,NaN,26,8.0,NaN
6,Checkout.com,40.0,2019-05-02,United Kingdom,London,Fintech,"Tiger Global Management, Insight Partners, DST...",2012.0,1830.00,NaN,15,4.0,NaN
7,Instacart,39.0,2014-12-30,United States,San Francisco,"Supply chain, logistics, & delivery","Khosla Ventures, Kleiner Perkins Caufield & By...",2012.0,2686.00,NaN,29,12.0,NaN
8,Databricks,38.0,2019-02-05,United States,San Francisco,Data management & analytics,"Andreessen Horowitz, New Enterprise Associates...",2013.0,3497.00,NaN,29,8.0,NaN
9,Revolut,33.0,2018-04-26,United Kingdom,London,Fintech,"index Ventures, DST Global, Ribbit Capital",2015.0,1716.00,NaN,31,6.0,NaN


In [12]:
top_20_companies = df.sort_values("Valuation ($B)", ascending=False)[:20]

fig4 = px.scatter(top_20_companies, x="Valuation ($B)", y="Total Raised(M)",
                 size="Investors Count", 
                 color=top_20_companies["Industry"].to_numpy(),
                 hover_name="Company", size_max=60, title="Top 20 companies")
fig4.update_layout(
    legend=dict(
        title="Industry"
    )
)

fig4.show()

In [69]:
top_10_countries = df[["Country", "Valuation ($B)"]].groupby(by="Country").sum()
top_10_countries = top_10_countries.sort_values(by="Valuation ($B)", ascending=False)[:10].reset_index()
top_10_countries_total_valuation = top_10_countries["Valuation ($B)"].sum()
top_10_countries_total_valuation_perc = top_10_countries_total_valuation * 100 / df["Valuation ($B)"].sum()
top_10_countries["iso_code"] = top_10_countries["Country"].apply(lambda x: pycountry.countries.lookup(x).alpha_3)
color_scale = ["#9BECB2", "#55E77C", "#26E1B2", "#25C488", "#06BE84", "#097759", "#003141", "#002837"]

fig7 = px.choropleth(top_10_countries, locations="iso_code", color="Valuation ($B)",
                    hover_name="Country", color_continuous_scale=color_scale,
                    title=f"Valuation for top 10 countries is {top_10_countries_total_valuation:.1f} B$ ({top_10_countries_total_valuation_perc:.1f}% of total)"
                    
                   )
fig7.update_layout(
    plot_bgcolor='#F3F6F8',  # Set the plot background color
    paper_bgcolor='#F3F6F8'  # Set the paper background color
)

fig7.show()

TypeError: choropleth() got an unexpected keyword argument 'opacity'

In [14]:
industry_total_val = df[["Industry", "Valuation ($B)"]].groupby(by="Industry").sum()

fig3 = go.Figure()
fig3.add_trace(go.Pie(labels=industry_total_val.index,
                     values=industry_total_val["Valuation ($B)"] 
                     )
             )
fig3.update_layout(title='Industries distribution by total valuation')
fig3.show()

In [61]:
industry_total_val = df[["Industry", "Valuation ($B)"]].groupby(by="Industry").sum().sort_values(by="Valuation ($B)",ascending=False)

fig3 = go.Figure()
fig3.add_trace(go.Bar(
    x=industry_total_val.index,
    y=industry_total_val["Valuation ($B)"],
    hovertext=industry_total_val,
    marker=dict(color='#3DED97')
    
))

fig3.update_layout(title='Industries distribution by total valuation')
fig3.show()


In [16]:
industry_total_val

,Valuation ($B)
Industry,
Fintech,833.87
Internet software & services,558.13
E-commerce & direct-to-consumer,331.09
Artificial Intelligence,321.19
Other,265.20
Health,182.52
"Supply chain, logistics, & delivery",175.76
Data management & analytics,135.84
Cybersecurity,122.70


In [32]:

total_valuation=round(df['Valuation ($B)'].sum()*10**-3,2)

3.41

In [38]:
toatl_number_unicorn=df.Company.count()
toatl_number_unicorn

1037

In [41]:
#toatl_funding
df["Total Raised(M)"].sum()*10**-3

583.25219

In [58]:
toatl_funding=round(df["Total Raised(M)"].sum()*10**-3,2)
toatl_funding

583.25

In [57]:
import math
total_valuation=math.ceil(df['Valuation ($B)'].sum()*10**-3)
total_valuation

4

In [44]:
total_valuation

'3 T'

In [45]:
import math

total_valuation = 3.41
total_valuation = f"{math.ceil(total_valuation)} T"
total_valuation

'4 T'